In [5]:
# -*- coding: utf-8 -*-
# Authors: Chitta Ranjan <cran2367@gmail.com>
#
# License: BSD 3 clause

In [152]:
import numpy as np
import pandas as pd
from itertools import chain
import warnings

In [400]:
class Sgt():
    '''
    Compute embedding of a single or a collection of discrete item 
    sequences. A discrete item sequence is a sequence made from a set
    discrete elements, also known as alphabet set. For example,
    suppose the alphabet set is the set of roman letters, 
    {A, B, ..., Z}. This set is made of discrete elements. Examples of
    sequences from such a set are AABADDSA, UADSFJPFFFOIHOUGD, etc.
    Such sequence datasets are commonly found in online industry,
    for example, item purchase history, where the alphabet set is
    the set of all product items. Sequence datasets are abundant in
    bioinformatics as protein sequences.
    Using the embeddings created here, classification and clustering
    models can be built for sequence datasets.
    Read more in https://arxiv.org/pdf/1608.03533.pdf

    Parameters
    ----------
    Input
    
    alphabets       Optional. The set of alphabets that make up all 
                    the sequences in the dataset. If not passed, the
                    alphabet set is automatically computed as the 
                    unique set of elements that make all the sequences.
    
    kappa           Tuning parameter, kappa > 0, to change the extraction of 
                    long-term dependency. Higher the value the lesser
                    the long-term dependency captured in the embedding.
                    Typical values for kappa are 1, 5, 10.
                   
    lengthsensitive Default false. This is set to true if the embedding of
                    should have the information of the length of the sequence.
                    If set to false then the embedding of two sequences with
                    similar pattern but different lengths will be the same.
                    lengthsensitive = false is similar to length-normalization.
    '''
    

    def __init__(self, alphabets=np.array([]), kappa=5, lengthsensitive=True):
        self.alphabets = alphabets
        self.kappa = kappa
        self.lengthsensitive = lengthsensitive
        
    def getpositions(self, sequence, alphabets):
        '''
        Compute index position elements in the sequence
        given alphabets set.

        Return list of tuples [(value, position)]
        '''
        positions = [(v, np.where(sequence==v)) for v in alphabets if v in sequence]
        
        return positions
    
    def fit(self, sequence, alphabets, lengthsensitive, kappa, flatten):
        '''
        Extract Sequence Graph Transform features using Algorithm-2.

        sequence            An array of discrete elements. For example,
                            np.array(["B","B","A","C","A","C","A","A","B","A"].
        alphabets           An array of the set of elements that make up the sequences.
                            For example, np.array(["A", "B", "C"].
        lengthsensitive     A boolean set to false by default.
        kappa               A positive value tuning parameter that is by default 1. 
                            Typically selected kappa from {1, 5, 10}.
        flatten

        return: sgt matrix

        '''
        
        size = alphabets.shape[0]
        l = 0
        W0, Wk = np.zeros((size,size)), np.zeros((size,size))
        positions = getpositions(sequence, alphabets)
        
        alphabets_in_sequence = np.unique(sequence)
        
        for i, u in enumerate(alphabets_in_sequence):

            U = np.array(positions[index][1]).ravel()

            for j, v in enumerate(alphabets_in_sequence):

                V2 = np.array(positions[index][1]).ravel()

                C = [(i,j) for i in U for j in V2 if j > i]

                W0[i,j] = len(C)

                cu = np.array([i[0] for i in C]) 
                cv = np.array([i[1] for i in C]) 

                Wk[i,j] = np.sum(np.exp(-kappa * np.abs(cu - cv)))

            l += U.shape[0]

        if lengthsensitive:
            W0 /= l

        W0[np.where(W0==0)] = 1e7  #avoid divide by 0

        sgt = np.power(np.divide(Wk, W0), 1/kappa)
        
        if(flatten):
            sgt = sgt.flatten()
        else:
            sgt = pd.DataFrame(sgt)
            sgt.columns = alphabets.tolist()
            sgt.index = alphabets.tolist()
        
        return sgt
    
    def __flatten(self, listOfLists):
        "Flatten one level of nesting"
        flat = [x for sublist in listOfLists for x in sublist]
        return flat

    def __estimate_alphabets(self, corpus):
        return(np.unique(np.asarray(self.__flatten(corpus))))
    
    def set_alphabets(self, corpus):
        self.alphabets = self.__estimate_alphabets(corpus)
        return self
        
    def fit_transform(self, corpus):
        '''
        corpus       A list of sequences. Each sequence is a list of alphabets.
        '''
        if(np.empty(self.alphabets)):
            self.alphabets = self.__estimate_alphabets(corpus)
            
        sgt = [self.fit(sequence=np.array(sequence), alphabets=self.alphabets, \
                          lengthsensitive=self.lengthsensitive, kappa=self.kappa, \
                          flatten = True).tolist() for sequence in corpus]

        return(np.array(sgt))

In [401]:
sgt = Sgt()

In [402]:
sequence = np.array(["B","B","A","C","A","C","A","A","B","A"])
corpus = [["B","B","A","C","A","C","A","A","B","A"], ["C", "Z", "Z", "Z", "D"]]
alphabets = np.array(["A", "B", "C"])
lengthsensitive = True
kappa = 5

In [403]:
# 
sgt.getpositions(sequence = np.array(corpus[0]), alphabets = alphabets)

[('A', (array([2, 4, 6, 7, 9]),)),
 ('B', (array([0, 1, 8]),)),
 ('C', (array([3, 5]),))]

In [404]:
sgt.fit(sequence, alphabets, lengthsensitive, kappa, flatten=False)

,A,B,C
A,0.369361,0.442463,0.537637
B,0.414884,0.468038,0.162774
C,0.454136,0.068693,0.214492


In [373]:
sgt.set_alphabets(corpus)

In [409]:
sgt.fit(sequence = np.array(corpus[1]), alphabets = np.array(["A","B","C","D","Z"]), lengthsensitive = lengthsensitive, kappa = kappa, flatten=False)

,A,B,C,D,Z
A,0.0,0.025271,0.408002,0.0,0.0
B,0.0,0.000000,0.000000,0.0,0.0
C,0.0,0.408002,0.468353,0.0,0.0
D,0.0,0.000000,0.000000,0.0,0.0
Z,0.0,0.000000,0.000000,0.0,0.0


In [410]:
sgt.fit(sequence = np.array(corpus[0]), alphabets = np.array(["A","B","C","D","Z"]), lengthsensitive = lengthsensitive, kappa = kappa, flatten=False)

,A,B,C,D,Z
A,0.369361,0.442463,0.537637,0.0,0.0
B,0.414884,0.468038,0.162774,0.0,0.0
C,0.454136,0.068693,0.214492,0.0,0.0
D,0.000000,0.000000,0.000000,0.0,0.0
Z,0.000000,0.000000,0.000000,0.0,0.0


In [343]:
s = sgt.fit_transform(corpus)
print(s)
# print(np.array(corpus[0]))

['A' 'B' 'C' 'D' 'Z']
True
5
[[0.36936141 0.44246287 0.53763711 0.         0.         0.41488439
  0.46803816 0.16277446 0.         0.         0.45413611 0.06869332
  0.21449197 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.02527063 0.40800217 0.         0.         0.
  0.         0.         0.         0.         0.         0.40800217
  0.4683531  0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]]


In [288]:
np.array(["B","B","A","C","A","C","A","A","B","A"])

array(['B', 'B', 'A', 'C', 'A', 'C', 'A', 'A', 'B', 'A'], dtype='<U1')

In [289]:
corpus[0]

['B', 'B', 'A', 'C', 'A', 'C', 'A', 'A', 'B', 'A']

In [242]:
np.array(corpus[0])

array(['B', 'B', 'A', 'C', 'A', 'C', 'A', 'A', 'B', 'A'], dtype='<U1')